In [7]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [8]:
# 데이터 불러오기 및 데이터 Augmentation, Normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'C:\\Users\\Smart\\Program\\gd1_project\\clothes_data\\clothes_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils
               .data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
# 모델 학습하기
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # train, val별로 학습    
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델 학습 모드
            else:
                model.eval()   # 모델 평가 모드

            running_loss = 0.0
            running_corrects = 0

            # 데이터 반복
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 매개변수 경사도를 0으로 설정
                optimizer.zero_grad()

                # 순전파
                # 학습 시에만 연산 기록을 추적
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 학습 단계인 경우 역전파 + 최적화
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 통계
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # 모델을 깊은 복사(deep copy)함
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # 가장 나은 모델 가중치 저장
    model.load_state_dict(best_model_wts)
    
    return model

In [10]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features

# nn.Linear(num_ftrs, len (class_names))로 일반화하기 .
model_ft.fc = nn.Linear(num_ftrs, len (class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# 모든 매개변수들이 최적화되었는지 관찰
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# 7 에폭마다 0.1씩 학습률 감소
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


In [6]:
# 학습 및 평가
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=12)

Epoch 0/11
----------
train Loss: 2.1681 Acc: 0.3299
val Loss: 1.9147 Acc: 0.4788

Epoch 1/11
----------
train Loss: 1.8629 Acc: 0.4251
val Loss: 2.5794 Acc: 0.4515

Epoch 2/11
----------
train Loss: 1.7032 Acc: 0.4721
val Loss: 1.8226 Acc: 0.5610

Epoch 3/11
----------
train Loss: 1.6104 Acc: 0.5092
val Loss: 1.8437 Acc: 0.5369

Epoch 4/11
----------
train Loss: 1.5201 Acc: 0.5226
val Loss: 1.9528 Acc: 0.5444

Epoch 5/11
----------
train Loss: 1.4585 Acc: 0.5371
val Loss: 1.7135 Acc: 0.5909

Epoch 6/11
----------
train Loss: 1.3372 Acc: 0.5822
val Loss: 1.7221 Acc: 0.5842

Epoch 7/11
----------
train Loss: 1.1181 Acc: 0.6544
val Loss: 1.4204 Acc: 0.6274

Epoch 8/11
----------
train Loss: 0.9882 Acc: 0.7007
val Loss: 1.4577 Acc: 0.6224

Epoch 9/11
----------
train Loss: 0.9656 Acc: 0.7022
val Loss: 1.4921 Acc: 0.6307

Epoch 10/11
----------
train Loss: 0.9143 Acc: 0.7133
val Loss: 1.4209 Acc: 0.6357

Epoch 11/11
----------
train Loss: 0.8869 Acc: 0.7232
val Loss: 1.4564 Acc: 0.6382

Tr